In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne

from utils import get_marker_name
from scipy import signal

In [2]:
# Loading EEG data

eeg     = np.loadtxt(r'C:\Users\emmac\OneDrive\Documents\GitHub\EEGTetris\dc\DSI-7_0210\devin_eeg.csv', delimiter=',', skiprows=1)
markers = np.loadtxt(r'C:\Users\emmac\OneDrive\Documents\GitHub\EEGTetris\dc\DSI-7_0210\devin_markers.csv', delimiter=',', skiprows=1)

Fs      = 300    # sampling rate of the DSI-7

In [ ]:
# Corrects for DC offset in EEG data

def nodc(eeg, Fs):                                  # Read as 'no DC'
    """ Corrects for DC offset in EEG data

    Parameters
    ----------
    eeg: np.arr
        EEG data where the no. of rows = # of sampled timepoints and data channels are across column
        1st channel is time (s)

    Fs: int
        sampling rate

    Returns
    -------
    eeg_nodc: np.arr
        og. EEG data arr corrected for DC offset across channels
    """

    fft      = np.fft.fft(eeg[:, 1:], axis=0)       # Need to exclude 1st channel so you don't 'DC un-offset' time
    frq      = np.fft.fftfreq(eeg.shape[0], 1/Fs)

    dc_idx   = np.abs(frq).argmin()   
    fft[dc_idx] = 0  

    ifft     = np.fft.ifft(fft, axis=0)
    eeg_nodc = np.hstack((eeg[:, 0].reshape(-1, 1), ifft))

    return eeg_nodc

In [ ]:
# MNE documentation (below)
# https://mne.tools/dev/generated/mne.filter.filter_data.html

# var_name = mne.filter.filter_data(t1_ndc.T[1:, :], sfreq=Fs, l_freq=8, h_freq=13)

In [3]:
# Epochs EEG data

def eeg_chop(eeg, markers):
    """ Epochs EEG data collected using the DSI-7 according to original (mislabeled) paradigm
    (ie. data collected on 02/10)

    Parameters
    ----------
    eeg: np.arr (n, 9)
        EEG data where n = # of timepoints and 9 channels of recording
        1st channel is time (s) and the remaining 8 are (in order): F3, Fz, F4, C3, C4, P3, P4, Gnd
            Gnd will read as column of 0s, can ignore

    markers: np.arr (m, 2) 
        m/2 = no. of trials
        Consists of corresponding marker #. Markers mark start+end of MI prompt presentation
        Marker encoding can be found under utils.py
        Functions (get_marker_name) found in utils.py can also decode marker # as needed

    Returns
    -------
    labels: np.arr (m/2, 1)
        processed markers arr to include only 'MI_TYPE' markers for each trial        

    baseline: np.arr (m/2, Fs * 2, 9)
        'baseline' epoch for each trial; consists of 2s (600 datapoints) before MI prompt
        9 channels like eeg arr

    motor_im: np.arr (m/2, Fs * 4, 9)
        active motor imagining epoch for each trial; consists of 4s (1200 datapoints) after MI prompt ends
        9 channels like eeg arr
    """
    eeg_f = eeg.copy()
    eeg_f[:, 0] = ['{:.3f}'.format(value) for value in eeg[:, 0]]
    
    marker_strt = markers[::2, :]
    marker_stop = markers[1::2, :]

    labels   = np.empty((marker_strt.shape[0], 1))
    baseline = np.empty((marker_strt.shape[0], 300 * 2, 9))
    motor_im = np.empty((marker_strt.shape[0], 300 * 4, 9))
    
    for i in range(marker_strt.shape[0]):
        end_idx = np.where(eeg[:, 0] <= marker_strt[0, 0])[0][-1]
        labels[i] = marker_strt[i, 1]
        baseline[i] = eeg[end_idx - 300 * 2:end_idx]

    for i in range(marker_stop.shape[0]):
        start_idx = np.where(eeg[:, 0] >= marker_stop[i, 0])[0][0]
        motor_im[i] = eeg[start_idx:start_idx + 300 * 4]

    return labels, baseline, motor_im